In [ ]:
import intake

### Intake GUI

> Use this as remote data https://s3.amazonaws.com/earth-data/UCMerced_LandUse/catalog.yml.
> Add this address in the `Remote` tab and select `UCMerced_LandUse_all`.

In [ ]:
# gui = intake.gui
# gui

In [ ]:
# gui.item #UCMerced_LandUse_all

In [ ]:
# da = gui.item.to_dask() # da : dataArray

### Another way of Catalog selection

In [ ]:
catalog_url = 'https://s3.amazonaws.com/earth-data/UCMerced_LandUse/catalog.yml'
da = intake.Catalog(catalog_url).UCMerced_LandUse_all.to_dask()

We can also directly specify the catalog by using `intake.Catalog`.

In [ ]:
land_dict = {land:i for i,land in enumerate(list(da.landuse.values))}

In [ ]:
da.values.shape

## Three Methods of generating Dynamic Map

### 1. Using Holoviews Dynamic Map

In [ ]:
import holoviews as hv
hv.extension('bokeh')

In [ ]:
def get_image(land,file):
    file = int(file)
    index = land_dict[land]
    return hv.Image(da.values[file][index])

# get_image(land = 'tenniscourt',file = 0)

In [ ]:
file_list = [str(i) for i in range(0,100)]
dmap = hv.DynamicMap(get_image,kdims = ['Land','File'])
dmap.redim.values(Land = list(da.landuse.values),File = file_list)

### 2. Using Panel, Streams and Params

In [ ]:
import panel as pn
from holoviews.streams import Params

In [ ]:
def get_image_from_file(land,file,cmap):
    index = land_dict[land]
    return hv.Image(da.values[file][index]).options(width=500, height=500,cmap = cmap)

#get_image_from_file('airplane',0,'viridis')

In [ ]:
files = pn.widgets.Select(name = 'File',value = 0 ,options=list(range(100)))
file_stream = Params(files,['value'],rename={'value': 'file'})

In [ ]:
lands = pn.widgets.Select(name = 'Land',value = 'airplane' ,options=list(da.landuse.values))
lands_stream = Params(lands,['value'],rename={'value': 'land'})

In [ ]:
cmaps = pn.widgets.Select(name = 'Color Map', value = 'dimgray',options=hv.plotting.list_cmaps() )
cmap_stream = Params(cmaps,['value'],rename = {'value':'cmap'})

In [ ]:
dmap = hv.DynamicMap(lambda land,file,cmap: get_image_from_file(land,file,cmap),
                     streams=[lands_stream,file_stream,cmap_stream])

In [ ]:
dashboard = pn.Column(
        pn.Column('# <pre> Land Explorer',
                  pn.Spacer(height=20)),
        pn.Row(
            dmap,
            pn.Spacer(width = 50),
            pn.Column(
               pn.Spacer(height= 50),
               cmaps,
               pn.Spacer(height= 20),
               lands,
               pn.Spacer(height= 20),
               files,
        )))

In [ ]:
dashboard

### 3. Using `param`

In [ ]:
import param

In [ ]:
class LandExplorer(param.Parameterized):
    
    cmap = param.ObjectSelector(default ='dimgray', objects=hv.plotting.list_cmaps())
    land = param.ObjectSelector(default='agricultural', objects=list(da.landuse.values))
    file =  param.ObjectSelector(default=0, objects = list(range(100)))
    
    @param.depends('cmap','land','file')
    def get_image_from_file(self):
        cmap = self.cmap
        land = self.land
        file = self.file
        index = land_dict[land]
        return hv.Image(da.values[file][index]).options(width=500, height=500,cmap = cmap)

In [ ]:
l = LandExplorer()

In [ ]:
dmap = pn.Row(hv.DynamicMap(l.get_image_from_file),l.param)

In [ ]:
dashboard = pn.Column(
    pn.Column('# <pre> Land Explorer',
              pn.Spacer(height=50)),
    pn.Row(
        pn.Row(dmap[0],
               pn.Spacer(width = 50),
               pn.Column(
                   pn.Spacer(height = 60),
                   dmap[1][1],
                   pn.Spacer(height = 30),
                   dmap[1][2],
                   pn.Spacer(height = 30),
                   dmap[1][3])
              )
        )
)

In [ ]:
dashboard.servable()

Deploy this dashboard from the CLI using:

```
 $ panel serve LandExplorer.ipynb
```